In [3]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from pyspark import SparkContext
import numpy

In [4]:
sc = SparkContext("local")

In [4]:
data.count()

120000

In [5]:
ratings = data.map(lambda l: l.split('|')[0:3])

In [6]:
ratings.first()

['199810', '208019', '0.0']

In [7]:
r = ratings.map(lambda l:Rating(int(l[0]),int(l[1]),float(l[2])))

In [8]:
r.first()

Rating(user=199810, product=208019, rating=0.0)

In [9]:
r.count()

120000

In [11]:
test_data = r.map(lambda l: (l[0],l[1]))

In [12]:
test_data.first()

(199810, 208019)

In [6]:
data = sc.textFile("/Users/shiyaoli/Data/NoBlank.txt")
ratings = data.map(lambda l: l.split('|')[0:3])
n= ratings.map(lambda l:Rating(int(l[0]),int(l[1]),float(l[2])))
bl_data = sc.textFile("/Users/shiyaoli/Data/Blank.txt")
b_ratings = bl_data.map(lambda l: l.split('|')[0:2])
b_data = b_ratings.map(lambda l:(int(l[0]),int(l[1])))

In [26]:
n_test_data = n.map(lambda l: (l[0],l[1]))

In [27]:
Error_min = 1000
for rank in [2,5,10,15,20,25,30]:
    for itr in [2,5,10,15,20,22]:
        model = ALS.train(n,rank,itr)
        predictions = model.predictAll(n_test_data).map(lambda r: ((r[0], r[1]), r[2])) 
        ratesAndPreds = n.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
        MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
        if MSE<Error_min:
            Error_min = MSE
            rank_min = rank
            itr_min = itr
        print("The rank is: ",rank)
        print("The iteration time is: ",itr)
        print("Mean Squared Error = " + str(MSE))       
print("The best rank is: ",rank_min)
print("The best iteration time is: ",itr_min)
print("The minimum of the MSE is: ",Error_min)


The rank is:  2
The iteration time is:  2
Mean Squared Error = 430.6972016190532
The rank is:  2
The iteration time is:  5
Mean Squared Error = 274.81897773290757
The rank is:  2
The iteration time is:  10
Mean Squared Error = 191.6107352860572
The rank is:  2
The iteration time is:  15
Mean Squared Error = 157.8078974348684
The rank is:  2
The iteration time is:  20
Mean Squared Error = 135.2756832759688
The rank is:  2
The iteration time is:  22
Mean Squared Error = 132.04882316010426
The rank is:  5
The iteration time is:  2
Mean Squared Error = 0.6851874458937123
The rank is:  5
The iteration time is:  5
Mean Squared Error = 0.08373323255840462
The rank is:  5
The iteration time is:  10
Mean Squared Error = 0.013941183145762575
The rank is:  5
The iteration time is:  15
Mean Squared Error = 0.0039864892965812
The rank is:  5
The iteration time is:  20
Mean Squared Error = 0.005217053838499128
The rank is:  5
The iteration time is:  22
Mean Squared Error = 0.0048681896021662095
The 

In [28]:
model = ALS.train(n,20,22)

In [50]:
nb = model.predictAll(b_data)

In [52]:
blk = nb.sortByKey().collect()
blk

[Rating(user=199810, product=242681, rating=4.328283794694013),
 Rating(user=199814, product=52519, rating=-11.554977178834742),
 Rating(user=199815, product=118841, rating=40.37049194452442),
 Rating(user=199815, product=129866, rating=-19.487839627792447),
 Rating(user=199816, product=274758, rating=-10.33282441537674),
 Rating(user=199819, product=214280, rating=1.9298185203833036),
 Rating(user=199820, product=292341, rating=27.081081300111368),
 Rating(user=199821, product=51933, rating=8.379489777368525),
 Rating(user=199821, product=42811, rating=1.5438507702141564),
 Rating(user=199822, product=185734, rating=-0.8255864726205624),
 Rating(user=199826, product=179298, rating=-0.0698098693636494),
 Rating(user=199827, product=92599, rating=44.641063575815835),
 Rating(user=199838, product=46697, rating=5.163624825133493),
 Rating(user=199839, product=266143, rating=-0.017225807150921417),
 Rating(user=199844, product=140229, rating=-2.7273598398648478),
 Rating(user=199844, produ

In [60]:
users = {}
for i in range(nb.count()):
    if blk[i][0] not in users:
        users[str(blk[i][0])] = {}
    users[str(blk[i][0])][str(blk[i][1])] = str(blk[i][2])
'242681' in users

True

In [63]:
users

{'199810': {'242681': '4.328283794694013'},
 '199814': {'52519': '-11.554977178834742'},
 '199815': {'129866': '-19.487839627792447'},
 '199816': {'274758': '-10.33282441537674'},
 '199819': {'214280': '1.9298185203833036'},
 '199820': {'292341': '27.081081300111368'},
 '199821': {'42811': '1.5438507702141564'},
 '199822': {'185734': '-0.8255864726205624'},
 '199826': {'179298': '-0.0698098693636494'},
 '199827': {'92599': '44.641063575815835'},
 '199838': {'46697': '5.163624825133493'},
 '199839': {'266143': '-0.017225807150921417'},
 '199844': {'77700': '-0.09972106953076021'},
 '199845': {'189985': '-31.34516417864537'},
 '199847': {'63072': '-26.685313760700605'},
 '199853': {'256134': '-6.405145672738552'},
 '199854': {'228397': '53.83969659513758'},
 '199855': {'69487': '0.17243988506703'},
 '199858': {'85596': '-25.874211018213202'},
 '199859': {'51667': '38.49780555540888'},
 '199862': {'280411': '-22.661484353263077'},
 '199863': {'241999': '-33.0982261487823'},
 '199868': {'2

In [61]:
SCORE_FILE = "/Users/shiyaoli/Data/Ranking3.txt"
OUT_FILE = "/Users/shiyaoli/Data/Final_Score.txt"

In [68]:
with open(OUT_FILE,'w') as output:
    with open(SCORE_FILE) as score:
        for line in score:
            if float(line.strip("\n").split("|")[2]) != 0.0:
                output.write(line)
            else:
                try:
                    output.write(line.strip("\n").split("|")[0]+"|"+line.strip("\n").split("|")[1]+"|"+str(users[line.strip("\n").split("|")[0]][line.strip("\n").split("|")[1]])+"\n")
                except KeyError:
                    output.write(line)